# Actividad Guiada 3
Estudiante: Efraín Paucar
https://github.com/EfrainPaucar/03-MAIR---Algoritmos-de-optimizacion-2019/blob/master/AG3/Efrain_Paucar_Castillo_AG3.ipynb

In [ ]:
import urllib.request

In [5]:
file = "swiss42.tsp"

In [6]:
urllib.request.urlretrieve("http://elib.zib.de/pub/mp-testdata/tsp/tsplib/tsp/swiss42.tsp",file)

('swiss42.tsp', <http.client.HTTPMessage at 0x9208ba8>)

In [7]:
import tsplib95
import random
from math import e

In [8]:
problem = tsplib95.load_problem(file)

#Nodos
Nodos = list(problem.get_nodes())

#Aristas
Aristas = list(problem.get_edges())

In [15]:
#Se genera una solución aleatoria con comienzo en el nodo 0
def crear_solucion(Nodos):
    solucion = [0]
    
    for i in range (len(Nodos)-1):
        solucion = solucion + [random.choice(list(set(Nodos)-set({0})-set(solucion)))]
    return solucion

#Devuelve la distancia entre 2 nodos
def distancia(a,b,problem):
    return problem.wfunc(a,b)

#Devuelve la distancia total de una trayectoria/solucion
def distancia_total(solucion,problem):
    distancia_total = 0
    for i in range(len(solucion)-1):
        distancia_total+=distancia(solucion[i],solucion[i+1],problem)
    return distancia_total + distancia(solucion[len(solucion)-1],solucion[0],problem)

In [16]:
#Búsqueda aleatoria
def busqueda_aleatoria(problem,N):
    #problem = datos del problema
    #N = número de iteraciones
    
    #Nodos
    Nodos = list(problem.get_nodes())
    
    mejor_solucion = []
    mejor_distancia = 10e100
    for i in range(N):
        solucion = crear_solucion(Nodos)
        distancia_solucion = distancia_total(solucion,problem)
        if distancia_solucion < mejor_distancia:
            mejor_solucion = solucion
            mejor_distancia = distancia_solucion
            
        print("la mejor solucion encontrada es: ", end="")
        print(solucion)
        print("con una distancia total de: ", end="")
        print(mejor_distancia)
        return mejor_solucion

In [17]:
busqueda_aleatoria(problem,10)

la mejor solucion encontrada es: [0, 10, 17, 28, 22, 38, 2, 27, 37, 23, 19, 26, 35, 7, 39, 25, 16, 41, 18, 1, 14, 20, 33, 36, 15, 12, 13, 8, 3, 29, 40, 24, 21, 34, 9, 32, 31, 4, 11, 30, 6, 5]
con una distancia total de: 4196


[0,
 10,
 17,
 28,
 22,
 38,
 2,
 27,
 37,
 23,
 19,
 26,
 35,
 7,
 39,
 25,
 16,
 41,
 18,
 1,
 14,
 20,
 33,
 36,
 15,
 12,
 13,
 8,
 3,
 29,
 40,
 24,
 21,
 34,
 9,
 32,
 31,
 4,
 11,
 30,
 6,
 5]

In [18]:
#Búsqueda loca, generador de vecindad
def genera_vecina(solucion):
    #Generador de soluciones vecinas:2-opt (intercambiar 2 nodos) Si hay N nodos se generan (N-1)x(N-2)/2 soluciones
    mejor_solucion = []
    mejor_distancia = 10e100
    for i in range(1,len(solucion)-1):
        for j in range(i+1,len(solucion)):
            vecina = solucion[:i] + [solucion[j]]+solucion[i+1:j]+[solucion[i]]+solucion[j+1:]
            distancia_vecina = distancia_total(vecina,problem)
            if distancia_vecina <= mejor_distancia:
                mejor_distancia = distancia_vecina
                mejor_solucion = vecina
    return mejor_solucion

In [20]:
#Búsqueda local:
def busqueda_local(problem,N):
    #problem = datos del problema
    #N = número de iteraciones
    mejor_solucion = []
    mejor_distancia = 10e100
    
    solucion_referencia = crear_solucion(Nodos)
    
    for i in range(N):
        vecina = genera_vecina(solucion_referencia)
        distancia_vecina = distancia_total(vecina,problem)
        if distancia_vecina <= mejor_distancia:
            mejor_solucion = vecina
            mejor_distancia = distancia_vecina
        
        solucion_referencia = vecina
        
        print("la mejor solucion encontrada es: ", end="")
        print(mejor_solucion)
        print("con una distancia total de: ", end="")
        print(mejor_distancia)
        return mejor_solucion


In [21]:
sol = busqueda_local(problem,100)

la mejor solucion encontrada es: [0, 18, 33, 24, 21, 22, 6, 19, 34, 26, 17, 4, 31, 30, 35, 29, 12, 8, 14, 2, 32, 38, 40, 27, 36, 16, 13, 39, 28, 1, 3, 11, 10, 9, 41, 25, 23, 7, 20, 5, 15, 37]
con una distancia total de: 4195


In [27]:
#Recocido Simulado - SA. 
#Función de probabilidad para aceptar peores soluciones
def probabilidad(T,d):
    r=random.random()
    if r >= (e**(-1*d)/((T*0.5*10**(-5)))):
        return True
    else:
        return False
    
#Función de descenso de temperatura
def bajar_Temperatura(T):
    return T*0.9

def recocido_simulado(problem, TEMPERATURA):
    #problem = datos del problema
    #T = Temperatura
    
    solucion_referencia = crear_solucion(Nodos)
    distancia_referencia = distancia_total(solucion_referencia,problem)
    
    mejor_solucion = []
    mejor_distancia = 10e100
    
    while TEMPERATURA > 1:
        print("#Temperatura:",TEMPERATURA, "Fitness:", mejor_distancia)
        #Genera una solucion vecina
        vecina = genera_vecina(solucion_referencia)
        
        #Calcula su valor(distancia)
        distancia_vecina = distancia_total(vecina,problem)
        
        #Si es la mejor solucion de todas se guarda
        if distancia_vecina < mejor_distancia:
            mejor_solucion = vecina
            mejor_distancia = distancia_vecina
            
        #Si la nueva vecina es mejor se cambia y si es peor se cambia según una probabilidad dependiente de T y de distancia_referencia - distancia
        if distancia_vecina < distancia_referencia or probabilidad(TEMPERATURA, abs(distancia_referencia - distancia_vecina)):
            solucion_referencia = vecina
            mejor_distancia = distancia_vecina
            
        TEMPERATURA = bajar_Temperatura(TEMPERATURA)

In [28]:
recocido_simulado(problem,10)

#Temperatura: 10 Fitness: 1e+101
#Temperatura: 9.0 Fitness: 4608
#Temperatura: 8.1 Fitness: 4262
#Temperatura: 7.29 Fitness: 3923
#Temperatura: 6.561 Fitness: 3592
#Temperatura: 5.9049000000000005 Fitness: 3386
#Temperatura: 5.3144100000000005 Fitness: 3189
#Temperatura: 4.7829690000000005 Fitness: 3024
#Temperatura: 4.3046721 Fitness: 2865
#Temperatura: 3.8742048900000006 Fitness: 2713
#Temperatura: 3.4867844010000004 Fitness: 2599
#Temperatura: 3.1381059609000004 Fitness: 2504
#Temperatura: 2.82429536481 Fitness: 2442
#Temperatura: 2.541865828329 Fitness: 2355
#Temperatura: 2.2876792454961 Fitness: 2301
#Temperatura: 2.05891132094649 Fitness: 2247
#Temperatura: 1.853020188851841 Fitness: 2194
#Temperatura: 1.6677181699666568 Fitness: 2148
#Temperatura: 1.5009463529699911 Fitness: 2106
#Temperatura: 1.350851717672992 Fitness: 2071
#Temperatura: 1.2157665459056928 Fitness: 2048
#Temperatura: 1.0941898913151236 Fitness: 2026
